<a href="https://colab.research.google.com/github/ypeng22/DLFinal/blob/main/image_cnn_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Mount Google Drive Data (If using Google Colaboratory)
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
except:
    print("Mounting Failed.")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
data_dir = "/content/gdrive/MyDrive/rgb_video_data"
model_dir = '/content/gdrive/MyDrive/cnn_lstm_epoch_'
#data_dir = "./rgb_video_data"
#model_dir = ""

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import os
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from torchvision.models import resnet101
import random
import imageio
import torch.nn.functional as F
subjects = ['S01', 'S02', 'S03', 'S04', 'S05','S06', 'S07', 'S08', 'S09', 'S10', 'S11', 'S12']
actions = ['A01', 'A02', 'A03', 'A04', 'A05','A06', 'A07', 'A08', 'A09', 'A10', 'A11']
reps = ['R01', 'R02', 'R03', 'R04', 'R05']

class BerkeleyMHAD(Dataset):

    def __init__(self, vid_names, root_dir, classes, transform=None):
        self.vid_names = vid_names # list of file names for videos (ex. S01_A01_R01)
        self.root_dir = root_dir # directory where videos are stored
        self.transform = transform
        self.classes = classes
        
    def __len__(self):
        return len(self.vid_names)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        path = os.path.join(self.root_dir, self.vid_names[idx])
        x = np.load(path)['x']
        
        data = []
        if self.transform:
            for i in range(x.shape[0]):
                data.append(self.transform(x[i]))
        
        vid_tensor = torch.stack(data)
        label = self.classes.index(np.load(path)['y']) 
        sample = {'x': vid_tensor, 'y': label}

        return sample
    
class cnn_lstm(nn.Module):
    def __init__(self, classes):
        super(cnn_lstm, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.conv3 = nn.Conv2d(32, 64, 3)
        self.conv4 = nn.Conv2d(64, 100, 3)
        self.pool1 = nn.MaxPool2d(3)
        self.n_hidden = 256
        self.n_layers = 3
        self.l_lstm = torch.nn.LSTM(input_size = 1500, 
                                 hidden_size = self.n_hidden,
                                 num_layers = self.n_layers, 
                                 batch_first = True, dropout=.3)
        self.fc1 = nn.Linear(1000, 300)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, classes)
        #self.relu = nn.LeakyReLU(.1)
        self.relu = nn.ReLU()

        self._initialize_weights()
        #self.resnet = resnet101(pretrained=True)
        #for param in self.resnet.parameters():
            #param.requires_grad = False
        #self.soft = nn.Softmax(dim = 0)

    def forward(self, x):
        batch = x.shape[0]
        #intialize lstm hidden state
        hidden_state = torch.zeros(self.n_layers, 1, self.n_hidden).to(dev)
        cell_state = torch.zeros(self.n_layers, 1, self.n_hidden).to(dev)
        #hidden_state = torch.randn(self.n_layers, 1, self.n_hidden).to(dev)
        #cell_state = torch.randn(self.n_layers, 1, self.n_hidden).to(dev)
        self.hidden = (hidden_state, cell_state)
        
        #x = self.pool1(self.relu(self.conv1(x)))
        #x = self.pool1(self.relu(self.conv2(x)))
        #x = self.pool1(self.relu(self.conv3(x)))
        #with torch.no_grad():
        #x1 = self.resnet(x)
        x1 = self.pool1(F.relu(self.conv1(x)))
        x1 = self.pool1(F.relu(self.conv2(x1)))
        x1 = self.pool1(F.relu(self.conv3(x1)))
        x1 = F.relu(self.conv4(x1))
        #print(x1.shape)
        #print(x.shape)
        x1 = x1.reshape(batch, -1).unsqueeze(0)
        #x1 = self.fc1(x1)
        lstm_out, _ = self.l_lstm(x1, self.hidden) #lstm_out shape is batch_size, seq len, hidden state
        lstm_out = lstm_out[:,-1,:]
        lstm_out = self.relu(self.fc2(lstm_out.squeeze()))
        lstm_out = self.fc3(lstm_out)
        #print(lstm_out.shape)
        #lstm_out = self.soft(lstm_out)
        return lstm_out

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out',
                                        nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight, mode='fan_out',
                                        nonlinearity='relu')
                nn.init.constant_(m.bias, 0)
    
def check(i):
    #insert more i.find terms for each action
    return i.find('A01') != -1 or i.find('A02') != -1 or i.find('A05') != -1 or i.find('A07') != -1
vid_names = [i for i in next(os.walk(data_dir))[2] if check(i)]

#vid_names = os.listdir(data_dir)
train_vid_names = [i for i in vid_names if i.find('S09') == -1 and i.find('S10') == -1 and i.find('S11') == -1 and i.find('S12') == -1]
valid_vid_names = [i for i in vid_names if i.find('S09') != -1 ]
test_vid_names = [i for i in vid_names if i.find('S10') != -1 or i.find('S11') != -1 or i.find('S12') != -1]

#data shape is (num_pics, height, width, channel)
Normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
train_dataset = BerkeleyMHAD(train_vid_names, data_dir, classes = np.arange(0, 11).tolist(), transform=transforms.Compose([transforms.ToTensor(), Normalize]))
valid_dataset = BerkeleyMHAD(valid_vid_names, data_dir, classes = np.arange(0, 11).tolist(), transform=transforms.Compose([transforms.ToTensor(), Normalize]))
test_dataset = BerkeleyMHAD(test_vid_names, data_dir, classes = np.arange(0, 11).tolist(), transform=transforms.Compose([transforms.ToTensor(), Normalize]))
#plt.imshow(np.transpose(train_dataset[0]['x'][0]))
print(train_dataset[0]['x'][0].shape)

batch_size = 1

dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#change 2 to number of classes

torch.Size([3, 240, 180])


In [4]:
#print(train_dataset[0]['x'].shape)
#imageio.mimwrite('output_filename.mp4', train_dataset[0]['x'].permute(0, 2, 3, 1).numpy() , fps = 22)
print(train_dataset[0]['x'].shape)

torch.Size([126, 3, 240, 180])


In [5]:
i = 0
'''for param in model.parameters():
  i += torch.numel(param)
  #print(param.)
print(i)'''

'for param in model.parameters():\n  i += torch.numel(param)\n  #print(param.)\nprint(i)'

In [6]:
model = cnn_lstm(4).to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()
epoch = 20
#train
shuffled = np.arange(len(train_dataset))

tguess = []
tanswers = []
tcorrect = 0
ttotal = 0
for t in range(epoch):
    train_loss = 0
    valid_loss = 0
    tguess = []
    tanswers = []
    tcorrect = 0
    ttotal = 0
    random.shuffle(shuffled)

    for i in range(len(train_dataset)):
        if i % 20 == 0:
          print(i)
        optimizer.zero_grad()
        torch.cuda.empty_cache()
        data = train_dataset[shuffled[i]]
        inpt = data['x'].to(dev)
        label = torch.tensor(data['y']).unsqueeze(0).to(dev)
        output = model(inpt).unsqueeze(0)
        loss = criterion(output, label) #.view(-1)
        loss.backward()
        #grads = []
        #for param in model.parameters():
        #    if param.grad is not None:
        #        grads.append(param.grad.view(-1))
        
        #grads = torch.cat(grads)
        #print(torch.sum(grads))
        #print(torch.norm(grads))
        optimizer.step()  
        #optimizer.zero_grad()
        train_loss += loss.item()
        
        if torch.argmax(output.squeeze()) == label:
            tcorrect += 1
        ttotal += 1
        tanswers.append(label.item())
        #tguess.append(output[0][1].item())
        tguess.append(torch.argmax(output.squeeze()))
        
    '''grads = []
    for param in model.parameters():
        if param.grad is not None:
            grads.append(param.grad.view(-1))
        
    grads = torch.cat(grads)
    print("norms: ", torch.norm(grads))'''
    
    torch.cuda.empty_cache()   

    '''with torch.no_grad():
        for i in range(len(valid_dataset)):
            torch.cuda.empty_cache()
            data = valid_dataset[i]
            inpt = data['x'].to(dev)
            label = torch.tensor(data['y']).unsqueeze(0).to(dev)
            output = model(inpt).unsqueeze(0)
            loss = criterion(output, label)
            valid_loss += loss.item()'''
    print("epoch:", t, " ", valid_loss / len(valid_dataset), train_loss / len(train_dataset))
    if t % 3 == 0:
        torch.save(model.state_dict(), model_dir + str(t) + '_state_dict.pt')



0


RuntimeError: ignored

In [ ]:
del output2
del loss

In [ ]:
'''model = cnn_lstm(11).to(dev)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.CrossEntropyLoss()'''

In [ ]:
#valid_dataset[i]['x'].shape

In [ ]:
#print(torch.cuda.memory_summary(dev))

In [ ]:
guess = []
answers = []
correct = 0
total = 0
dev = torch.device("cuda:0")
model = model.to(dev)
#with torch.no_grad():
for i in range(len(test_dataset)):
    if i % 10 == 0:
      print(i)
    data = test_dataset[i]
    inpt = data['x'].to(dev)
    label = torch.tensor(data['y']).to(dev)
    output2 = model(inpt)
    if torch.argmax(output2.squeeze()) == label:
        correct += 1
    total += 1
    answers.append(label.item())
    guess.append(output2[1].item())
    del output2
    torch.cuda.empty_cache()
#guess = torch.argmax(guess.squeeze(), dim=1)
#guess = np.array(guess).squeeze()
print("train accuracy = " + str(tcorrect / ttotal))
print("test accuracy = " + str(correct / total))
#print('CNN AUC: %.4f' % roc_auc_score(testlabel, guess), ' AUPRC: %.4f' % average_precision_score(testlabel, guess))
#torch.save(model, '/content/gdrive/MyDrive/cnn_lstm.torch')

In [ ]:
from torch.autograd import Variable
class GradientAttack():
        
        """
        
        """
        
        def __init__(self, loss, epsilon):
            """
            
            """
            self.loss = loss
            self.epsilon = epsilon

        def forward(self, x, y_true, model):
            """
            
            """
            # TODO: Implement a one step gradient attack
            # Step 1: Calculate the Loss and then calculate the
            # gradient of the Loss w.r.t the image
            y_true = torch.Tensor([y_true]).cuda()
            loss = self.loss(model(x).unsqueeze(0), y_true.long())
            #x.requires_grad = True
            loss.backward()
            # Step 2: Add the gradient (or its sign for each pixel),
            # multiplied by a small step size, to 
            # the original image
            # you might need to clamp the modified image to 
            # make sure the values of each pixel are between [0,1]
            x = x + torch.sign(x.grad) * self.epsilon
            return x

In [ ]:
adv_attack = GradientAttack(criterion, 0.1)
guess = []
answers = []
correct = 0
total = 0
dev = torch.device('cuda:0')
model = model.to(dev)

for i in range(len(test_dataset)):
    data = test_dataset[i]
    inpt = data['x'].to(dev)
    inpt2 = Variable(inpt, requires_grad=True)
    label = torch.tensor(data['y']).to(dev)
    inpt3 = adv_attack.forward(inpt2, label, model)
    output3 = model(inpt3)
    if torch.argmax(output3.squeeze()) == label:
        correct += 1
    total += 1
    answers.append(label.item())
    guess.append(output3[1].item())
    del output3
    torch.cuda.empty_cache()
#guess = torch.argmax(guess.squeeze(), dim=1)
#guess = np.array(guess).squeeze()
print("train accuracy = " + str(tcorrect / ttotal))
print("test accuracy = " + str(correct / total))

In [ ]:
print(torch.cuda.memory_summary(dev))